The purpose of this code is to generate random rough surface to simulate the rough surface in additive manufacturing.

updated on **2025/03/31**

* The random seed for generating the roughness profiles will be saved
* Not only the height but also the first-order derivatives along X- and Y-axes will be saved

updated on: **2025.03.08**

* Larger rough surface was created with more control points
* Then strcutured grids was cropped from the large rough surface

updated on: **2025.03.01**

* Choose gmsh_env
* gmsh has been installed by 'pip install gmsh'

updated on: **2025.02.20**

* The float is randomized in the range of [-1,+1]
* The Ra is integrated and then scaled and shifted to form a body.
* Three independent boundary conditions (pure extension along X-, Y-axes and pure shear in XY plane)

updated on: **2025.02.07**

* The range has been adjusted to [0,1]
* The control points and the sampling points for structured grids are not overlapped

updated on: **2025.01.22**


* Maybe the script can be transformed into class-based (NOT implemented yet)
* GMSH is a module **NOT** a class

updated on: **2025.01.21**

* most code in the cell are packed into functions
* the these function 'def' will be called by a loop to generated random model and related input files
* the number for the random seed should be printed (**NOT** possible according to ChatGPT)

updated on: **2024.12.23**

* rough surface is simulated by spline algorithm
* three dimensional RVE is generated by GMSH
* the rough volume will be simulated in ABAQUS
* fatigue indicator parameter will be extracted
* the rough surface is equivalent to grayscale images
* convolutional neural network will be used thereafter

Reference

[[1] Spline example](https://scipython.com/book/chapter-8-scipy/examples/two-dimensional-interpolation-with-scipyinterpolaterectbivariatespline/)

[[2] GMSH example](https://www.geeksforgeeks.org/how-to-generate-mesh-in-python-with-gmsh-module/)

[[3] GMSH example](https://jsdokken.com/src/tutorial_gmsh.html)

[[4] ABAQUS face identifier](https://docs.software.vt.edu/abaqusv2024/English/SIMACAEELMRefMap/simaelm-r-3delem.htm)

# Install and import necessary toolbox

Import the libraries to generate random surface



In [17]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import RectBivariateSpline
import os
from scipy.spatial import KDTree
import pandas as pd
import time
import sys
import gmsh

Link to the Google Drive

In [18]:
abaqus_inp_files_folder = 'C:\\Abaqus_Works\\roughness_simulation\\abaqus_inp_files'
input_rough_surface_folder = 'C:\\Abaqus_Works\\roughness_simulation\\input_rough_surfaces'
gmsh_data_files_folder = 'C:\\Abaqus_Works\\roughness_simulation\\gmsh_data_files'
rand_seed_file_folder = 'C:\\Abaqus_Works\\roughness_simulation'

Generate the seed number for the file (only executed once)

In [19]:
# # Generate 100,000 random integers in the range [0, 2**32 - 1]
# num_ids = 100000
# rand_integers = np.random.randint(0, 2**32, size=num_ids, dtype=np.uint32)

# # Save the integers into a CSV file without a header
# rand_seed_file = "random_seed.csv"

# rand_seed_path = os.path.join(rand_seed_file_folder, rand_seed_file)
# np.savetxt(rand_seed_path, rand_integers, fmt='%d', delimiter=',')

# print(f"Generated {num_ids} random integers and saved them to {rand_seed_file} without a column name.")

# Determine the size and control points of 2-D spline

*   Define the range of the volume on X-Y plane

    [Smaller size will be cropped form this volume]
*   Define the interval of control points along X and Y axes

    [Denser points will be interpolated by the fitted spline]
*   Define the thickness of the plate
*   Define Ra in unit 'mm'
*   If necessary, the range can be adjusted into [0.1, 0.5] with increment of 0.005.(not necessary any more 2025/02/20)
* By doing this, ***even*** number of structured grids can be achieved (not necessary any more 2025/02/20)


In [20]:
#-------------------------------------------------------------------------------
# Define original size and parameters about the control points
#-------------------------------------------------------------------------------
# define the range for the point series
range_low_control_points = -1.8 # (mm)
range_upp_control_points = +1.8 # (mm)

# Define the interval of control points in the unit of 'mm'
interval_control_points = 0.060 # unit of 'mm' about half of the hatching distance
number_control_points = int((range_upp_control_points - range_low_control_points)/interval_control_points) + 1
print(f'The number of control points along one direction is {number_control_points}')

x_1D_control_points = np.linspace(range_low_control_points, range_upp_control_points, number_control_points)
y_1D_control_points = np.linspace(range_low_control_points, range_upp_control_points, number_control_points)
print(x_1D_control_points)

#-------------------------------------------------------------------------------
# Define the structured grids which should be denser or smaller than control points
#-------------------------------------------------------------------------------
range_low_struct_grids = 0.0  # (mm)
# range_upp_struct_grids = 0.318  # (mm)
range_upp_struct_grids = 0.638  # (mm)
# range_upp_struct_grids = 1.276  # (mm)

# Number of the cropped structured grids along X- or Y-axes
num_crop_single_edge = 0
print(f"{num_crop_single_edge} grids are cropped from the structured grids along one edge.")

if range_low_struct_grids < range_low_control_points or range_upp_struct_grids > range_upp_control_points:
    print(f"Something wrong about the range of structured grids and control points.")

#-------------------------------------------------------------------------------
# Define thickness and roughness Ra
#-------------------------------------------------------------------------------
# Define the plate thickness in the unit of mm
thickness = 0.1 # unit of mm
# Ra in the unit of mm
Ra = 0.005      # unit of mm

print(f'The thickness of the volume and roughness are {thickness} mm {Ra} mm respectively.')

#-------------------------------------------------------------------------------
# Define the structured grids which should be denser than control points
#-------------------------------------------------------------------------------
# Define the interval of structured grid which is denser than the control points
elem_sub_times = 1.0
interval_basis = 0.004
interval_struct_grids = interval_basis/elem_sub_times #300.0/147.0*0.001/elem_sub_times   #   with unit of 'mm'
print(f"The interval of the structured grids is {interval_struct_grids} mm")

# Given the control size (1.8 mm)x(1.8 mm) is quite larger than the struct-grid,
#   random shift will be introduced. Here the range for randomizing an integer will be computed.
low_lim = round((range_low_control_points - range_low_struct_grids)/interval_struct_grids)
upp_lim = round((range_upp_control_points - range_upp_struct_grids)/interval_struct_grids)

# Number of the structured grids along X- or Y-axes
number_struct_grids = round((range_upp_struct_grids - range_low_struct_grids)/interval_struct_grids) + 1
print(f"number of structured grids along one direction is {number_struct_grids}")

#-------------------------------------------------------------------------------
# Define the range of the structured grids
# Structured grids are not necessary to be overlapped with control points
#-------------------------------------------------------------------------------
x_1D_struct_grids = np.linspace(range_low_struct_grids, range_upp_struct_grids, number_struct_grids)
y_1D_struct_grids = np.linspace(range_low_struct_grids, range_upp_struct_grids, number_struct_grids)
X_2D_struct_grids, Y_2D_struct_grids = np.meshgrid(x_1D_struct_grids, y_1D_struct_grids)

print(x_1D_struct_grids)
print(len(x_1D_struct_grids))

# dense grids to measure the roughness
interval_dense_points = 0.002   # unit of mm
number_dense_grids = int((range_upp_control_points - range_low_control_points)/interval_dense_points) + 1
x_1D_dense_grids = np.linspace(range_low_control_points, range_upp_control_points, number_dense_grids)
y_1D_dense_grids = np.linspace(range_low_control_points, range_upp_control_points, number_dense_grids)
# print(x_1D_dense_grids)

The number of control points along one direction is 61
[-1.8  -1.74 -1.68 -1.62 -1.56 -1.5  -1.44 -1.38 -1.32 -1.26 -1.2  -1.14
 -1.08 -1.02 -0.96 -0.9  -0.84 -0.78 -0.72 -0.66 -0.6  -0.54 -0.48 -0.42
 -0.36 -0.3  -0.24 -0.18 -0.12 -0.06  0.    0.06  0.12  0.18  0.24  0.3
  0.36  0.42  0.48  0.54  0.6   0.66  0.72  0.78  0.84  0.9   0.96  1.02
  1.08  1.14  1.2   1.26  1.32  1.38  1.44  1.5   1.56  1.62  1.68  1.74
  1.8 ]
0 grids are cropped from the structured grids along one edge.
The thickness of the volume and roughness are 0.1 mm 0.005 mm respectively.
The interval of the structured grids is 0.004 mm
number of structured grids along one direction is 161
[0.        0.0039875 0.007975  0.0119625 0.01595   0.0199375 0.023925
 0.0279125 0.0319    0.0358875 0.039875  0.0438625 0.04785   0.0518375
 0.055825  0.0598125 0.0638    0.0677875 0.071775  0.0757625 0.07975
 0.0837375 0.087725  0.0917125 0.0957    0.0996875 0.103675  0.1076625
 0.11165   0.1156375 0.119625  0.1236125 0.1276    

Define the cropped structured grids parameters

In [21]:
#-------------------------------------------------------------------------------
# Define the cropped structured grids
#-------------------------------------------------------------------------------
number_crop_struct_grids = number_struct_grids - 2*num_crop_single_edge
print(f"number of cropped structured grids along one direction is {number_crop_struct_grids}")

effective_crop_length = (number_crop_struct_grids-1)*interval_struct_grids
print(f"the effective length of the cropped structured grids is {round(effective_crop_length*1000)} um")

# Crop the structured grids into smaller size around the center
if (number_crop_struct_grids > number_struct_grids) or ((number_struct_grids-number_crop_struct_grids)%2 != 0):
    print("Error in cropping the structured grids!")

# crop the grids symmetrically about the center
if number_struct_grids == number_crop_struct_grids:
    index_start = 0
    index_stop = number_struct_grids
else:
    index_start = (number_struct_grids - number_crop_struct_grids)//2
    index_stop = - index_start

x_1D_crop_struct_grids = x_1D_struct_grids[index_start:index_stop]
y_1D_crop_struct_grids = y_1D_struct_grids[index_start:index_stop]
print(x_1D_crop_struct_grids)
print(len(x_1D_crop_struct_grids))

#-------------------------------------------------------------------------------
# Define the element sizes for GMSH meshing
#-------------------------------------------------------------------------------
# mesh size of the characteristic length which will determine the element size
size_elem_fine = interval_struct_grids  # unit of mm
size_elem_rough = 0.02                  # unit of mm
print(f"fine and rough element size are {size_elem_fine} and {size_elem_rough}")

#-------------------------------------------------------------------------------
# Define the Young's moduli and Poisson's ratio and tensile strain amplitude
#-------------------------------------------------------------------------------
# volume young moduli, Poisson's ratio and shear moduli
volume_young_moduli = 121000.0
volume_possion_ratio = 0.308
volume_shear_moduli = volume_young_moduli/(2.0*(1.0 + volume_possion_ratio))

# tensile strain amplitude
nom_stress = 100.0 # unit of MPa
# strain_amp = 0.005
# print(f"The nominal strain is {strain_amp} mm/mm")

number of cropped structured grids along one direction is 161
the effective length of the cropped structured grids is 640 um
[0.        0.0039875 0.007975  0.0119625 0.01595   0.0199375 0.023925
 0.0279125 0.0319    0.0358875 0.039875  0.0438625 0.04785   0.0518375
 0.055825  0.0598125 0.0638    0.0677875 0.071775  0.0757625 0.07975
 0.0837375 0.087725  0.0917125 0.0957    0.0996875 0.103675  0.1076625
 0.11165   0.1156375 0.119625  0.1236125 0.1276    0.1315875 0.135575
 0.1395625 0.14355   0.1475375 0.151525  0.1555125 0.1595    0.1634875
 0.167475  0.1714625 0.17545   0.1794375 0.183425  0.1874125 0.1914
 0.1953875 0.199375  0.2033625 0.20735   0.2113375 0.215325  0.2193125
 0.2233    0.2272875 0.231275  0.2352625 0.23925   0.2432375 0.247225
 0.2512125 0.2552    0.2591875 0.263175  0.2671625 0.27115   0.2751375
 0.279125  0.2831125 0.2871    0.2910875 0.295075  0.2990625 0.30305
 0.3070375 0.311025  0.3150125 0.319     0.3229875 0.326975  0.3309625
 0.33495   0.3389375 0.342925  0.

# Generate random 2D surface by scipy library

* Generate random surface at the control points
* Fit the random control points with spline function
* The number of control points will determines the number of peaks and valleys
* Sampling the dense structured grids with the available spline interpolator



In [22]:
def generate_crop_random_structured_grids(id_surf, rand_seed, b_seed):

    # print(f"Deterministic seed is NOT used!")
    if b_seed:
        print(f"Deterministic seed is used!")
        np.random.seed(rand_seed)
    else:
        print(f"Deterministic seed is NOT used!")

    # np.random.seed(rand_surf_id)

    # Create random values for the control points
    Z_2D_control_points = np.random.uniform(-1, +1, size=(number_control_points, number_control_points))    # it is uniform distribution
    # print(Z_2D_control_points.shape)
    # Z_2D_control_points = np.random.rand(number_control_points, number_control_points)
    # Z_2D_control_points = np.ones((number_control_points, number_control_points))

    # fitting the control points and create the B-spline interpolator
    random_spline_surface = RectBivariateSpline(x_1D_control_points, y_1D_control_points, Z_2D_control_points)

    #************************************************************************************************************************
    # calculate the roughness with the dense grid with the spacing of 1um
    #************************************************************************************************************************
    Z_2D_dense_grids = random_spline_surface(x_1D_dense_grids, y_1D_dense_grids)

    mean_dense_grids = np.mean(Z_2D_dense_grids)

    # shift the surface by mean_surface
    shifted_mean_dense_grids = Z_2D_dense_grids - mean_dense_grids

    # calculate the Ra (or average of the absolute mean)
    mean_abs_shifted_dense_grids = np.mean(np.abs(shifted_mean_dense_grids))
    print(f"mean value and Ra of the dense sampling is {mean_dense_grids} and {mean_abs_shifted_dense_grids}")
    #************************************************************************************************************************

    # Z = np.ones((number_struct_grids, number_struct_grids)) + thickness
    # Sampling at the structured grids
    if low_lim >= upp_lim:
        x_shift_grid = 0
        y_shift_gird = 0
    else:
        # x_shift_grid = -282*int(elem_sub_times)
        # y_shift_gird = -260*int(elem_sub_times)
        x_shift_grid = np.random.randint(low_lim, upp_lim)
        y_shift_gird = np.random.randint(low_lim, upp_lim)

        print(f"x shift {x_shift_grid} and y shift {y_shift_gird}")

    x_1D_sample_grid = x_1D_struct_grids + x_shift_grid*interval_struct_grids
    y_1D_sample_grid = y_1D_struct_grids + y_shift_gird*interval_struct_grids

    print(f'the sampling range in the X or Y axis')
    print(x_1D_sample_grid)

    # extract the dR/dx and dR/dy and save them
    baseline_surface    = random_spline_surface(x_1D_sample_grid, y_1D_sample_grid)
    dx_baseline_surface = random_spline_surface(x_1D_sample_grid, y_1D_sample_grid, dx=1, dy=0)
    dy_baseline_surface = random_spline_surface(x_1D_sample_grid, y_1D_sample_grid, dx=0, dy=1)

    scaled_shifted_baseline_surface = (baseline_surface - mean_dense_grids)/mean_abs_shifted_dense_grids*Ra
    dx_scaled_baseline_surface = dx_baseline_surface/mean_abs_shifted_dense_grids*Ra
    dy_scaled_baseline_surface = dy_baseline_surface/mean_abs_shifted_dense_grids*Ra

    # shift the scaled surface to form a volume
    Z_2D_struct_grids = scaled_shifted_baseline_surface + thickness
    print(f"The shape of Z_2D_struct_grids is {Z_2D_struct_grids.shape}")
    print(f"max {np.max(Z_2D_struct_grids)} min {np.min(Z_2D_struct_grids)}")
    # print(f"make sure Ra is close to {np.max(Z_2D_struct_grids) - np.min(Z_2D_struct_grids)} (mm)")

    # crop the grids symmetrically about the center
    if number_struct_grids == number_crop_struct_grids:
        index_start = 0
        index_stop = number_struct_grids
    else:
        index_start = (number_struct_grids - number_crop_struct_grids)//2
        index_stop = - index_start

    print(f"index start {index_start} index stop {index_stop} in for the cropped structured grids")

    Z_2D_crop_struct_grids = Z_2D_struct_grids[index_start:index_stop, index_start:index_stop]

    # roughness values with mean at Z=0
    R_2D_crop_struct_grids = scaled_shifted_baseline_surface[index_start:index_stop, index_start:index_stop]
    dx_2D_crop_struct_grids = dx_scaled_baseline_surface[index_start:index_stop, index_start:index_stop]
    dy_2D_crop_struct_grids = dy_scaled_baseline_surface[index_start:index_stop, index_start:index_stop]

    # write the Z-coordinates on the cropped structured grids into the file
    # effective_crop_length = (number_crop_struct_grids-1)*interval_struct_grids
    # export the 
    # file_name_input_csv = f"input_surface_L{round((range_upp_struct_grids - range_low_struct_grids)*1000)}um_C{num_crop_single_edge}_S{int(elem_sub_times)}_ID{id_surf:0{5}d}.csv"
    # rough_surface_file_path_csv = os.path.join(input_rough_surface_folder, file_name_input_csv)
    # df_crop_struct_grids = pd.DataFrame(R_2D_crop_struct_grids)
    # df_crop_struct_grids.to_csv(rough_surface_file_path_csv, index=False, header=False)
    
    #----------------------------------
    file_name_input_xlsx = f"input_surface_L{round((range_upp_struct_grids - range_low_struct_grids)*1000)}um_C{num_crop_single_edge}_S{int(elem_sub_times)}_ID{id_surf:0{5}d}.xlsx"
    rough_surface_file_path = os.path.join(input_rough_surface_folder, file_name_input_xlsx)

    sheet_name_list = ['R', 'dRdx', 'dRdy']

    with pd.ExcelWriter(rough_surface_file_path, engine='openpyxl') as writer:
        # output the R
        sheet_name = f"{sheet_name_list[0]}"
        df = pd.DataFrame(R_2D_crop_struct_grids[:, :])
        df.to_excel(writer, sheet_name=sheet_name, index=False, header=False)

        # output the dx
        sheet_name = f"{sheet_name_list[1]}"
        df = pd.DataFrame(dx_2D_crop_struct_grids[:, :])
        df.to_excel(writer, sheet_name=sheet_name, index=False, header=False)

        # output the dy
        sheet_name = f"{sheet_name_list[2]}"
        df = pd.DataFrame(dy_2D_crop_struct_grids[:, :])
        df.to_excel(writer, sheet_name=sheet_name, index=False, header=False)

    return x_1D_crop_struct_grids, y_1D_crop_struct_grids, Z_2D_crop_struct_grids #, input_roughness_file_path

# Generate mesh by GMSH

* Function 'add_line' to register the lines into a dictionary and assign unique tag to each line

* Function 'get_line_tag' to retrieve the tage by giving two points


In [23]:
# Add lines and register them in the dictionary
def add_line(p1, p2, line_lookup):
    # sort the two points according to its number
    if p1 < p2:
        p_A = p1
        p_B = p2
    else:
        p_A = p2
        p_B = p1
    # construct tuple of two points on both sides of a line
    line = (p_A, p_B)
    if line in line_lookup:
        return False
    else:
        tag = len(line_lookup) + 1  # Unique tag
        gmsh.model.geo.addLine(p_A, p_B, tag)
        line_lookup[line] = tag  # Store with sorted points as key
        return True

# Retrieve a line tag by two points and direction
def get_line_tag(p1, p2, line_lookup):
    direction = 0
    if p1 < p2:
        p_A = p1
        p_B = p2
        direction = +1
    else:
        p_A = p2
        p_B = p1
        direction = -1
    line = (p_A, p_B)

    return line_lookup.get(line)*direction

Generate mesh using GMSH from the random surface


In [24]:
def generate_mesh_using_gmsh(x_1D_crop_struct_grids, y_1D_crop_struct_grids, Z_2D_crop_struct_grids, rand_surf_id):
    #-----------------------------------------------------------------------
    # Initialize gmsh
    #-----------------------------------------------------------------------
    gmsh.initialize()
    gmsh.model.add("rough_surface_model")

    #-----------------------------------------------------------------------
    # register points to the model
    #-----------------------------------------------------------------------
    node_number_surface = 0
    # register the cropped structured grids points
    node_index_surface = np.zeros((number_crop_struct_grids, number_crop_struct_grids), dtype=int)
    for i in range(number_crop_struct_grids):
        for j in range(number_crop_struct_grids):
            node_number_surface += 1
            node_index_surface[i, j] = int(node_number_surface)
            # print(node_number_surface)
            # last element in geo.add_point is tag of the point
            gmsh.model.geo.add_point(x_1D_crop_struct_grids[i], y_1D_crop_struct_grids[j], Z_2D_crop_struct_grids[i, j], 
                                     size_elem_fine, node_number_surface)

    # register four corner points on the bottom plane
    # form the lines and curve on the bottom face
    p_bottom_1 = node_number_surface + 1
    p_bottom_2 = node_number_surface + 2
    p_bottom_3 = node_number_surface + 3
    p_bottom_4 = node_number_surface + 4

    # four coner points on the plane of Z=0
    gmsh.model.geo.add_point(x_1D_crop_struct_grids[0], y_1D_crop_struct_grids[0], 0.0, size_elem_rough, p_bottom_1)
    gmsh.model.geo.add_point(x_1D_crop_struct_grids[-1], y_1D_crop_struct_grids[0], 0.0, size_elem_rough, p_bottom_2)
    gmsh.model.geo.add_point(x_1D_crop_struct_grids[-1], y_1D_crop_struct_grids[-1], 0.0, size_elem_rough, p_bottom_3)
    gmsh.model.geo.add_point(x_1D_crop_struct_grids[0], y_1D_crop_struct_grids[-1], 0.0, size_elem_rough, p_bottom_4)

    #-----------------------------------------------------------------------
    # register all lines necessary to form the surfaces
    # all lines will be used to form the wireframe
    #-----------------------------------------------------------------------
    # loop through the grids to generate surface mesh
    # because the four points might not be on the same plane,
    #  it is better to
    #     p4|     |p3
    #   ----|--- -|-------
    #       | + 2 |
    #       | 1 + |
    #   ----|-----|-------
    #     p1|     |p2
    #   one square element

    line_lookup = {}

    for i in range(number_crop_struct_grids-1):
        for j in range(number_crop_struct_grids-1):
            # extract four points in the grid
            p1 = node_index_surface[i,     j    ]
            p2 = node_index_surface[i + 1, j    ]
            p3 = node_index_surface[i + 1, j + 1]
            p4 = node_index_surface[i,     j + 1]

            # 202504.09
            # seperate the square element into two triangles
            # P1->P2->P4->P1
            add_line(p1, p2, line_lookup)
            add_line(p2, p4, line_lookup)
            add_line(p4, p1, line_lookup)
            # P2->P3->P4->P2
            add_line(p2, p3, line_lookup)
            add_line(p3, p4, line_lookup)
            add_line(p4, p2, line_lookup)
            

    # add lines on the bottom plane
    #     y /^\
    #      p4|    p3
    #    ----|-----|-------
    #        | +   |
    #        |   + |
    #    ----|-----|-----------> x
    #      p1|    p2
    #
    add_line(p_bottom_1, p_bottom_2, line_lookup)
    add_line(p_bottom_2, p_bottom_3, line_lookup)
    add_line(p_bottom_3, p_bottom_4, line_lookup)
    add_line(p_bottom_4, p_bottom_1, line_lookup)

    # add four vertical lines connecting the curve surface to the bottom plane
    add_line(p_bottom_1, node_index_surface[0, 0], line_lookup)
    add_line(p_bottom_2, node_index_surface[-1, 0], line_lookup)
    add_line(p_bottom_3, node_index_surface[-1, -1], line_lookup)
    add_line(p_bottom_4, node_index_surface[0, -1], line_lookup)

    #-----------------------------------------------------------------------
    # register closed curves and faces of the top curve surface
    #-----------------------------------------------------------------------
    curve_loop_number = 0
    face_number_surface = 0
    face_list =[]

    """
     [top rough surface]
    """
    for i in range(number_crop_struct_grids-1):
        for j in range(number_crop_struct_grids-1):
            # extract four points in the grid
            p1 = node_index_surface[i,    j   ]
            p2 = node_index_surface[i + 1, j   ]
            p3 = node_index_surface[i + 1, j + 1]
            p4 = node_index_surface[i,    j + 1]

            # 2025.04.09
            # seperate the square element into two triangles
            # P1->P2->P4->P1
            line_1_2 = get_line_tag(p1, p2, line_lookup)
            line_2_4 = get_line_tag(p2, p4, line_lookup)
            line_4_1 = get_line_tag(p4, p1, line_lookup)
            curve_loop_number += 1  # the tag of the closed curve
            gmsh.model.geo.addCurveLoop([line_1_2, line_2_4, line_4_1], curve_loop_number)
            # form the face
            face_number_surface += 1
            gmsh.model.geo.addPlaneSurface([curve_loop_number], face_number_surface)
            face_list.append(face_number_surface)

            # P1->P2->P4->P1
            line_2_3 = get_line_tag(p2, p3, line_lookup)
            line_3_4 = get_line_tag(p3, p4, line_lookup)
            line_4_2 = get_line_tag(p4, p2, line_lookup)
            curve_loop_number += 1  # the tag of the closed curve
            gmsh.model.geo.addCurveLoop([line_2_3, line_3_4, line_4_2], curve_loop_number)
            # form the face
            face_number_surface += 1
            gmsh.model.geo.addPlaneSurface([curve_loop_number], face_number_surface)
            face_list.append(face_number_surface)

    # Step 3: Assign a Physical Group to the Surface
    # rough_surface_group_tag = gmsh.model.addPhysicalGroup(2, surface_list)  # "2" indicates a surface
    # gmsh.model.setPhysicalName(2, rough_surface_group_tag, "rough_surface")

    """
     [bottom plane]
    """
    line_1_2 = get_line_tag(p_bottom_1, p_bottom_2, line_lookup)
    line_2_3 = get_line_tag(p_bottom_2, p_bottom_3, line_lookup)
    line_3_4 = get_line_tag(p_bottom_3, p_bottom_4, line_lookup)
    line_4_1 = get_line_tag(p_bottom_4, p_bottom_1, line_lookup)
    curve_loop_number += 1  # the tag of the closed curve
    gmsh.model.geo.addCurveLoop([line_1_2, line_2_3, line_3_4, line_4_1], curve_loop_number)

    face_number_surface += 1
    gmsh.model.geo.addPlaneSurface([curve_loop_number], face_number_surface)

    face_list.append(face_number_surface)

    """
     [left plane]
    """
    line_list_face = []
    for i in range(number_crop_struct_grids - 1):
        p1 = node_index_surface[0, i]
        p2 = node_index_surface[0, i + 1]

        line = get_line_tag(p1, p2, line_lookup)
        line_list_face.append(line)

    line = get_line_tag(node_index_surface[0, -1], p_bottom_4, line_lookup)
    line_list_face.append(line)

    line = get_line_tag(p_bottom_4, p_bottom_1, line_lookup)
    line_list_face.append(line)

    line = get_line_tag(p_bottom_1, node_index_surface[0, 0], line_lookup)
    line_list_face.append(line)

    curve_loop_number += 1
    gmsh.model.geo.addCurveLoop(line_list_face, curve_loop_number)

    face_number_surface += 1
    gmsh.model.geo.addPlaneSurface([curve_loop_number], face_number_surface)

    face_list.append(face_number_surface)

    """
     [right plane]
    """
    line_list_face = []
    for i in range(number_crop_struct_grids - 1):
        p1 = node_index_surface[-1, i]
        p2 = node_index_surface[-1, i + 1]

        line = get_line_tag(p1, p2, line_lookup)
        line_list_face.append(line)

    line = get_line_tag(node_index_surface[-1, -1], p_bottom_3, line_lookup)
    line_list_face.append(line)

    line = get_line_tag(p_bottom_3, p_bottom_2, line_lookup)
    line_list_face.append(line)

    line = get_line_tag(p_bottom_2, node_index_surface[-1, 0], line_lookup)
    line_list_face.append(line)

    curve_loop_number += 1
    gmsh.model.geo.addCurveLoop(line_list_face, curve_loop_number)

    face_number_surface += 1
    gmsh.model.geo.addPlaneSurface([curve_loop_number], face_number_surface)

    face_list.append(face_number_surface)

    """
     [front plane]
    """
    line_list_face = []
    for i in range(number_crop_struct_grids - 1):
        p1 = node_index_surface[i, 0]
        p2 = node_index_surface[i + 1, 0]

        line = get_line_tag(p1, p2, line_lookup)
        line_list_face.append(line)

    line = get_line_tag(node_index_surface[-1, 0], p_bottom_2, line_lookup)
    line_list_face.append(line)

    line = get_line_tag(p_bottom_2, p_bottom_1, line_lookup)
    line_list_face.append(line)

    line = get_line_tag(p_bottom_1, node_index_surface[0, 0], line_lookup)
    line_list_face.append(line)

    curve_loop_number += 1
    gmsh.model.geo.addCurveLoop(line_list_face, curve_loop_number)

    face_number_surface += 1
    gmsh.model.geo.addPlaneSurface([curve_loop_number], face_number_surface)

    face_list.append(face_number_surface)

    """
     [back plane]
    """
    line_list_face = []
    for i in range(number_crop_struct_grids - 1):
        p1 = node_index_surface[i, -1]
        p2 = node_index_surface[i + 1, -1]

        line = get_line_tag(p1, p2, line_lookup)
        line_list_face.append(line)

    line = get_line_tag(node_index_surface[-1, -1], p_bottom_3, line_lookup)
    line_list_face.append(line)

    line = get_line_tag(p_bottom_3, p_bottom_4, line_lookup)
    line_list_face.append(line)

    line = get_line_tag(p_bottom_4, node_index_surface[0, -1], line_lookup)
    line_list_face.append(line)

    curve_loop_number += 1
    gmsh.model.geo.addCurveLoop(line_list_face, curve_loop_number)

    face_number_surface += 1
    gmsh.model.geo.addPlaneSurface([curve_loop_number], face_number_surface)

    face_list.append(face_number_surface)

    #-----------------------------------------------------------------------
    # generate mesh according to the registered entities
    #-----------------------------------------------------------------------
    # Create the relevant Gmsh data structures from Gmsh model.
    gmsh.model.geo.addSurfaceLoop(face_list, 1)
    gmsh.model.geo.addVolume([1])

    # volume_tag = gmsh.model.addPhysicalGroup(3, [1])
    # gmsh.model.setPhysicalName(3, volume_tag, "rough_volume")

    gmsh.option.setNumber("Mesh.Optimize", 1)  # 启用优化器
    gmsh.option.setNumber("Mesh.OptimizeNetgen", 1)  # 启用 Netgen 优化
    gmsh.option.setNumber("Mesh.QualityType", 2)  # 设置网格质量类型为最小雅可比值
    gmsh.option.setNumber("Mesh.Smoothing", 10)  # 平滑操作次数
    gmsh.option.setNumber("Mesh.CharacteristicLengthMin", 0.001)  # 最小单元大小
    gmsh.option.setNumber("Mesh.CharacteristicLengthMax", size_elem_rough)   # 最大单元大小

    gmsh.model.geo.synchronize()

    # Generate mesh (only volume 3D elements)
    gmsh.model.mesh.generate(3)

    # Remove 1D elements
    line_tags = gmsh.model.getEntities(dim=1)  # Get all 1D entities
    for dim, tag in line_tags:
        gmsh.model.mesh.removeElements(dim, tag)

    # # Remove 2D elements
    # surface_tags = gmsh.model.getEntities(dim=2)  # Get all 2D entities
    # for dim, tag in surface_tags:
    #     gmsh.model.mesh.removeElements(dim, tag)

    #-----------------------------------------------------------------------
    # write the nodes and elements into inp file
    #-----------------------------------------------------------------------
    effective_crop_length = (number_crop_struct_grids-1)*interval_struct_grids
    model_name = f"gmsh_L{round((range_upp_struct_grids - range_low_struct_grids)*1000)}um_C{num_crop_single_edge}_S{int(elem_sub_times)}_ID{rand_surf_id:0{5}d}"

    # format_msh = "msh"
    # file_name = f"{model_name}.{format_msh}"
    # gmsh.write(file_name)
    # files.download(file_name)

    format_inp = "inp"
    file_name_gmsh = f"{model_name}.{format_inp}"
    path_gmsh_data_files = os.path.join(gmsh_data_files_folder, file_name_gmsh)
    gmsh.write(path_gmsh_data_files)
    # files.download(file_name)

    # file_path = os.path.join(drive_path, file_name)
    # gmsh.write(file_path)

    # Finalize Gmsh
    gmsh.finalize()

    return path_gmsh_data_files

# Generate ABAQUS 'inp' file

Parse the node and element sections from GMSH generated 'inp' file


In [25]:
def parse_nodes(lines):
    """
    Parse the node data from the input file.
    This assumes the node data starts with "*NODE" and contains node definitions.
    """
    nodes = []
    in_nodes_section = False
    for line in lines:
        line = line.upper() # convert the string into uppercase

        if len(line) > 1 and line.startswith('**'):
            continue

        if line.startswith('*NODE'):
            in_nodes_section = True
        elif in_nodes_section:
            if line.startswith('*'):
                in_nodes_section = False
                break  # End of nodes section
            node_data = list(map(float, line.split(',')))
            nodes.append(node_data[1:]) # node number is not included because they are already sorted.

    return nodes


Parse the volume and surface elements from GMSH generated 'inp' file


In [26]:
def parse_elements(lines):
    """
    Parse the element data from the input file.
    This assumes the element data starts with "*ELEMENT" and contains node definitions.
    The first integer is the element number
    """
    elements_volume = []
    elements_surface = []

    in_elements_section = False
    volume_element = True

    index_volume_element = 0

    for line in lines:
        line = line.upper() # convert the string into uppercase

        if len(line) > 1 and line.startswith('**'):
            continue

        if line.startswith('*ELEMENT'):
            in_elements_section = True
            if 'CPS3' in line:
                volume_element = False
            if 'C3D4' in line:
                volume_element = True
        elif in_elements_section:
            if line.startswith('*'):
                in_elements_section = False
                break  # End of elements section
            element_data = list(map(int, line.split(',')))
            if volume_element:
                index_volume_element += 1
                element_data[0] = index_volume_element
                elements_volume.append(element_data[:])   # element number is included
            else:
                elements_surface.append(element_data[:])   # element number is included

    return elements_volume, elements_surface

Exclude the surface elements on four vertical plane and bottom plane so that only surface elements on top will be saved

In [27]:
def parse_top_surface_elements(elements_surface, nodes, range_low_crop_struct_grids, range_upp_crop_struct_grids):
    """
     Differentiate the CPS3 elements into six sets (bottom, left, right, front, back, top-rough)
    """
    tol = 1E-6

    elements_surface_Z_top = []

    # loop through each element
    for element in elements_surface:

        x = [nodes[node_number-1][0] for node_number in element[1:]]
        y = [nodes[node_number-1][1] for node_number in element[1:]]
        z = [nodes[node_number-1][2] for node_number in element[1:]]

        if abs(max(x) - min(x)) < tol:
            # along X-axis
            if abs(min(x) - range_low_crop_struct_grids) < tol:
                pass
            elif abs(max(x) - range_upp_crop_struct_grids) < tol:
                pass
            else:
                elements_surface_Z_top.append(element)

        elif abs(max(y) - min(y)) < tol:
            # along Y-axis
            if abs(min(y) - range_low_crop_struct_grids) < tol:
                pass
            elif abs(max(y) - range_upp_crop_struct_grids) < tol:
                pass
            else:
                elements_surface_Z_top.append(element)

        elif abs(max(z) - min(z)) < tol and abs(min(z) - 0) < tol:
            pass
        else:
            # along Z-axis top plane
            elements_surface_Z_top.append(element)

    return elements_surface_Z_top

Classify the top C3D4 elements neighbour free top surface into four categories as S1, S2, S3, S4 with definition in [ABAQUS](https://docs.software.vt.edu/abaqusv2024/English/SIMACAEELMRefMap/simaelm-r-3delem.htm)

In [28]:
def volume_element_nodes_on_top_surface(elements_surface, tri_elem):

    for element in elements_surface:
        if sorted(tri_elem) == sorted(element[1:]):
            return True
    return False

def volume_elements_neighboring_top_surface(elements_volume, elements_surface):

    cps3_in_c3d4_s1 = []
    cps3_in_c3d4_s2 = []
    cps3_in_c3d4_s3 = []
    cps3_in_c3d4_s4 = []

    nodes_set_surface = set()  # use set to accelerate the set
    # form the node set from surface cps3 elements without duplicated node
    for element in elements_surface:
        nodes_set_surface.update(element[1:])  # update the set

    # find the c3d4 elements which contain nodes on the surface cps3
    for elem_3d in elements_volume:
        nodes_included = [node for node in elem_3d[1:] if node in nodes_set_surface]

        if len(nodes_included) == 3:  # only consider elements with exactly 3 common nodes
            face = sorted(nodes_included)

            # Check which face of the c3d4 element the cps3 surface corresponds to
            if face == sorted([elem_3d[1], elem_3d[2], elem_3d[3]]):
                cps3_in_c3d4_s1.append(elem_3d[0])
            elif face == sorted([elem_3d[1], elem_3d[2], elem_3d[4]]):
                cps3_in_c3d4_s2.append(elem_3d[0])
            elif face == sorted([elem_3d[2], elem_3d[3], elem_3d[4]]):
                cps3_in_c3d4_s3.append(elem_3d[0])
            else:
                cps3_in_c3d4_s4.append(elem_3d[0])

        if len(nodes_included) == 4:  # only consider elements with exactly 4 common nodes
            print(f"The mesh has low quality!")
            if volume_element_nodes_on_top_surface(elements_surface, [elem_3d[1], elem_3d[2], elem_3d[3]]):
                cps3_in_c3d4_s1.append(elem_3d[0])
            if volume_element_nodes_on_top_surface(elements_surface, [elem_3d[1], elem_3d[2], elem_3d[4]]):
                cps3_in_c3d4_s2.append(elem_3d[0])
            if volume_element_nodes_on_top_surface(elements_surface, [elem_3d[2], elem_3d[3], elem_3d[4]]):
                cps3_in_c3d4_s3.append(elem_3d[0])
            if volume_element_nodes_on_top_surface(elements_surface, [elem_3d[1], elem_3d[3], elem_3d[4]]):
                cps3_in_c3d4_s4.append(elem_3d[0])

    return nodes_set_surface, cps3_in_c3d4_s1, cps3_in_c3d4_s2, cps3_in_c3d4_s3, cps3_in_c3d4_s4

Write ABAQUS inp file according to its format

In [ ]:
def write_numbers_with_increment(list_of_integer, increment, file):
    # 16 items per line
    for i in range(0, len(list_of_integer), increment):
        # retrieve certain amount of items
        chunk = list_of_integer[i:i + increment]
        # seperate the items by comma
        file.write(",".join(map(str, chunk)) + "\n")

def write_inp_file(nodes_volume, elements_volume, nodes_coat, elements_coat, nodes_volume_Z_top,
        node_volume_VZ_X0_Y0, node_volume_VZ_X1_Y0, node_volume_VZ_X0_Y1, node_volume_VZ_X1_Y1,
        nodes_volume_EX_Y0_Z0, nodes_volume_EX_Y1_Z0, nodes_volume_EY_X0_Z0, nodes_volume_EY_X1_Z0,
        nodes_volume_EZ_X0_Y0, nodes_volume_EZ_X0_Y1, nodes_volume_EZ_X1_Y0, nodes_volume_EZ_X1_Y1,
        nodes_volume_X_positive, nodes_volume_X_negative, nodes_volume_Y_positive, nodes_volume_Y_negative, nodes_volume_Z_bottom,
        elements_volume_top_s1, elements_volume_top_s2, elements_volume_top_s3, elements_volume_top_s4, file_name_abaqus):
    

    #-----------------------------------------------------------------------
    #   find the structured nodes on the top surface of the volume 
    #   and aggregate them into a node set
    #-----------------------------------------------------------------------    
    nodes_XY_free_volume_surface = []
    for node_id in nodes_volume_Z_top:
        node = nodes_volume[node_id-1]
        nodes_XY_free_volume_surface.append([node[0], node[1]])

    nodes_XY_free_volume_surface = np.array(nodes_XY_free_volume_surface)
    tree = KDTree(nodes_XY_free_volume_surface)

     # create nodes list for the cropped structured grid points
    nodes_XY_crop_struct_grids = []
    # here to select the necessary structured grids
    # for row in range(0,number_crop_struct_grids, int(elem_sub_times)):
    #     for col in range(0,number_crop_struct_grids, int(elem_sub_times)):
    for row in range(0,number_crop_struct_grids, 1):
        for col in range(0,number_crop_struct_grids, 1):
            node_XY = [x_1D_crop_struct_grids[row], y_1D_crop_struct_grids[col]]
            nodes_XY_crop_struct_grids.append(node_XY)
    
    # register the structured grids to the nodes in the free mesh
    nodes_struct_volume_surface = []
    for idx_struct, node_struct in enumerate(nodes_XY_crop_struct_grids):
        dist, idx_free = tree.query([node_struct])
        if idx_free is None or len(idx_free) == 0:
            print(f"KDTree query failed for node {node_struct}")
        # Updated on 2025/04/10
        nodes_struct_volume_surface.append(nodes_volume_Z_top[idx_free[0]])

    with open(file_name_abaqus, 'w') as file:
        #---------------------------------------------------------------
        # *Heading and Preprint
        #---------------------------------------------------------------
        # write *Heading and *Preprent
        file.write('*Heading\n')
        file.write(f'**Job name: {file_name_abaqus}\n')
        file.write('*Preprint, echo=NO, model=NO, history=NO, contact=NO\n')
        file.write('**\n')

        #---------------------------------------------------------------
        # *PART 'COAT'-membrane
        #---------------------------------------------------------------
        part_coat = 'COAT'
        file.write(f'*Part, name={part_coat}\n')

        # Node
        file.write('*Node\n')
        for index_node, node in enumerate(nodes_coat):
            file.write(f'{index_node +1}, {node[0]:.9f}, {node[1]:.9f}, {node[2]:.9f}\n')

        # Only when the cropping from sturctured grid is **ZERO**
        if num_crop_single_edge == 0:
            # all node in the coat but only need X and Y coordinates
            nodes_coat_array = np.array(nodes_coat)
            nodes_free_xy = nodes_coat_array[:,0:2]

            # create K-D Tree
            tree = KDTree(nodes_free_xy)

            # create nodes list for the control points
            nodes_XY_control_points = []

            for row in range(number_control_points):
                for col in range(number_control_points):
                    node_XY = [x_1D_control_points[row], y_1D_control_points[col]]
                    nodes_XY_control_points.append(node_XY)
            
            # register the control points to the nodes in the free mesh
            nodes_set_control_points = []

            for idx_control, node_control in enumerate(nodes_XY_control_points):
                dist, idx_free = tree.query([node_control])
                if idx_free is None or len(idx_free) == 0:
                    print(f"KDTree query failed for node {node_control}")
                # original operation
                nodes_set_control_points.append(idx_free[0] + 1)


            # create nodes list for the structured grid points
            nodes_XY_sparse_struct_grids_coat = []
            # here to select the necessary structured grids
            for row in range(0,number_crop_struct_grids,5):
                for col in range(0,number_crop_struct_grids,5):
                    node_XY = [x_1D_crop_struct_grids[row], y_1D_crop_struct_grids[col]]
                    nodes_XY_sparse_struct_grids_coat.append(node_XY)

            # register the structured grids to the nodes in the free mesh
            nodes_set_sparse_struct_grids_coat = []
            for idx_struct, node_struct in enumerate(nodes_XY_sparse_struct_grids_coat):
                dist, idx_free = tree.query([node_struct])
                if idx_free is None or len(idx_free) == 0:
                    print(f"KDTree query failed for node {node_struct}")
                # origianl operation
                nodes_set_sparse_struct_grids_coat.append(idx_free[0] + 1)


            
        # Element
        file.write('*Element, Type=M3D3\n')
        for element in elements_coat:
            file.write(f'{element[0]}, {element[1]}, {element[2]}, {element[3]}\n')

        # define element set for the membrane
        elset_coat_sect = 'SET_COAT_SECT'
        file.write(f'*Elset, elset={elset_coat_sect}, generate\n')
        file.write(f'1, {len(elements_coat)}, 1\n')

        # define section
        material_coat = 'ELAS_COAT'
        file.write(f'*Membrane Section, elset={elset_coat_sect}, material={material_coat}\n')
        file.write(f'0.001,\n')

        # end of part COAT
        file.write('*End Part\n')

        #---------------------------------------------------------------
        # *PART Volume
        #---------------------------------------------------------------
        # writing volume part: C3D4 elements
        part_volume = 'VOLUME'
        file.write(f'*Part, name={part_volume}\n')

        # Node
        file.write('*Node\n')
        for index_node, node in enumerate(nodes_volume):
            file.write(f'{index_node + 1}, {node[0]:.9f}, {node[1]:.9f}, {node[2]:.9f}\n')

        # Element
        file.write('*Element, Type=C3D4\n')
        for element in elements_volume:
            file.write(f'{element[0]}, {element[1]}, {element[2]}, {element[3]}, {element[4]}\n')

        # node set at structured grids on the top surface
        nset_z_struct_volume_surface = 'F_VOL_SURF_STRUCT'
        file.write(f'*Nset, nset={nset_z_struct_volume_surface}\n')
        write_numbers_with_increment(nodes_struct_volume_surface, 16, file)

        # define element set for the volume
        elset_volume = 'SET_VOLUME_SECT'
        file.write(f'*Elset, elset={elset_volume}, generate\n')
        file.write(f'1, {len(elements_volume)}, 1\n')

        # define section for the volume
        material_volume = 'VOLUME_ELAS'
        file.write(f'*Solid Section, elset={elset_volume}, material={material_volume}\n')
        file.write(f',\n')

        # end part volume
        file.write('*End Part\n')

        #---------------------------------------------------------------
        # Assembly
        #---------------------------------------------------------------
        file.write('*Assembly, name=Assembly\n')
        file.write(f'*Instance, name={part_coat+"-1"}, part={part_coat}\n')
        file.write('*End Instance\n')
        file.write(f'*Instance, name={part_volume+"-1"}, part={part_volume}\n')
        file.write('*End Instance\n')

        # Only when the cropping from sturctured grid is **ZERO**
        if num_crop_single_edge == 0:
            nset_coat_control_points = 'NSET_COAT_CONTROL_POINTS'
            file.write(f'*Nset, nset={nset_coat_control_points}, instance={part_coat+"-1"}\n')
            write_numbers_with_increment(nodes_set_control_points, 16, file)

            nset_coat_struct_grids = 'NSET_COAT_SPARSE_STRUCT_GRIDS'
            file.write(f'*Nset, nset={nset_coat_struct_grids}, instance={part_coat+"-1"}\n')
            write_numbers_with_increment(nodes_set_sparse_struct_grids_coat, 16, file)

        nset_vz_x0_y0 = 'VZ_X0_Y0'
        nset_vz_x1_y0 = 'VZ_X1_Y0'
        nset_vz_x0_y1 = 'VZ_X0_Y1'
        nset_vz_x1_y1 = 'VZ_X1_Y1'

        file.write(f'*Nset, nset={nset_vz_x0_y0}, instance={part_volume+"-1"}\n')
        file.write(f'{node_volume_VZ_X0_Y0}\n')
        file.write(f'*Nset, nset={nset_vz_x1_y0}, instance={part_volume+"-1"}\n')
        file.write(f'{node_volume_VZ_X1_Y0}\n')
        file.write(f'*Nset, nset={nset_vz_x0_y1}, instance={part_volume+"-1"}\n')
        file.write(f'{node_volume_VZ_X0_Y1}\n')
        file.write(f'*Nset, nset={nset_vz_x1_y1}, instance={part_volume+"-1"}\n')
        file.write(f'{node_volume_VZ_X1_Y1}\n')

        nset_ex_y0_z0 = 'EX_Y0_Z0'
        nset_ex_y1_z0 = 'EX_Y1_Z0'
        nset_ey_x0_z0 = 'EY_X0_Z0'
        nset_ey_x1_z0 = 'EY_X1_Z0'
        nset_ez_x0_y0 = 'EZ_X0_Y0'
        nset_ez_x0_y1 = 'EZ_X0_Y1'
        nset_ez_x1_y0 = 'EZ_X1_Y0'
        nset_ez_x1_y1 = 'EZ_X1_Y1'

        file.write(f'*Nset, nset={nset_ex_y0_z0}, instance={part_volume+"-1"}\n')
        write_numbers_with_increment(nodes_volume_EX_Y0_Z0, 16, file)

        file.write(f'*Nset, nset={nset_ex_y1_z0}, instance={part_volume+"-1"}\n')
        write_numbers_with_increment(nodes_volume_EX_Y1_Z0, 16, file)

        file.write(f'*Nset, nset={nset_ey_x0_z0}, instance={part_volume+"-1"}\n')
        write_numbers_with_increment(nodes_volume_EY_X0_Z0, 16, file)

        file.write(f'*Nset, nset={nset_ey_x1_z0}, instance={part_volume+"-1"}\n')
        write_numbers_with_increment(nodes_volume_EY_X1_Z0, 16, file)

        file.write(f'*Nset, nset={nset_ez_x0_y0}, instance={part_volume+"-1"}\n')
        write_numbers_with_increment(nodes_volume_EZ_X0_Y0, 16, file)

        file.write(f'*Nset, nset={nset_ez_x0_y1}, instance={part_volume+"-1"}\n')
        write_numbers_with_increment(nodes_volume_EZ_X0_Y1, 16, file)

        file.write(f'*Nset, nset={nset_ez_x1_y0}, instance={part_volume+"-1"}\n')
        write_numbers_with_increment(nodes_volume_EZ_X1_Y0, 16, file)

        file.write(f'*Nset, nset={nset_ez_x1_y1}, instance={part_volume+"-1"}\n')
        write_numbers_with_increment(nodes_volume_EZ_X1_Y1, 16, file)

        # node sets for boundary conditions
        nset_x_negative_face = 'F_X0'
        nset_x_positive_face = 'F_X1'
        nset_y_negative_face = 'F_Y0'
        nset_y_positive_face = 'F_Y1'
        nset_z_bottom_face = 'F_Z0'
        
        file.write(f'*Nset, nset={nset_x_negative_face}, instance={part_volume+"-1"}\n')
        write_numbers_with_increment(nodes_volume_X_negative, 16, file)

        file.write(f'*Nset, nset={nset_x_positive_face}, instance={part_volume+"-1"}\n')
        write_numbers_with_increment(nodes_volume_X_positive, 16, file)

        file.write(f'*Nset, nset={nset_y_negative_face}, instance={part_volume+"-1"}\n')
        write_numbers_with_increment(nodes_volume_Y_negative, 16, file)

        file.write(f'*Nset, nset={nset_y_positive_face}, instance={part_volume+"-1"}\n')
        write_numbers_with_increment(nodes_volume_Y_positive, 16, file)

        file.write(f'*Nset, nset={nset_z_bottom_face}, instance={part_volume+"-1"}\n')
        write_numbers_with_increment(nodes_volume_Z_bottom, 16, file)

        

        # element sets to define faces of solid element which match with coat
        elset_rough_top_s1 = 'ROUGH_TOP_ELEM_S1'
        elset_rough_top_s2 = 'ROUGH_TOP_ELEM_S2'
        elset_rough_top_s3 = 'ROUGH_TOP_ELEM_S3'
        elset_rough_top_s4 = 'ROUGH_TOP_ELEM_S4'

        if elements_volume_top_s1:
            file.write(f'*Elset, elset={elset_rough_top_s1}, internal, instance={part_volume+"-1"}\n')
            write_numbers_with_increment(elements_volume_top_s1, 16, file)

        if elements_volume_top_s2:
            file.write(f'*Elset, elset={elset_rough_top_s2}, internal, instance={part_volume+"-1"}\n')
            write_numbers_with_increment(elements_volume_top_s2, 16, file)

        if elements_volume_top_s3:
            file.write(f'*Elset, elset={elset_rough_top_s3}, internal, instance={part_volume+"-1"}\n')
            write_numbers_with_increment(elements_volume_top_s3, 16, file)

        if elements_volume_top_s4:
            file.write(f'*Elset, elset={elset_rough_top_s4}, internal, instance={part_volume+"-1"}\n')
            write_numbers_with_increment(elements_volume_top_s4, 16, file)

        # define TIE interaction between COAT and VOLUME
        surface_volume_rough_top = 'M_SURF_VOLUME_ROUGH_TOP'
        file.write(f'*Surface, type=ELEMENT, name={surface_volume_rough_top}\n')
        if elements_volume_top_s1:
            file.write(f'{elset_rough_top_s1}, S1\n')

        if elements_volume_top_s2:
            file.write(f'{elset_rough_top_s2}, S2\n')

        if elements_volume_top_s3:
            file.write(f'{elset_rough_top_s3}, S3\n')

        if elements_volume_top_s4:
            file.write(f'{elset_rough_top_s4}, S4\n')

        # define the surface of the COAT
        elset_coat_tie = 'COAT_ELEM_SNEG'
        file.write(f'*Elset, elset={elset_coat_tie}, internal, instance={part_coat+"-1"}, generate\n')
        file.write(f'1, {len(elements_coat)}, 1\n')

        surface_coat = 'S_SURF_COAT_SNEG'
        file.write(f'*Surface, type=ELEMENT, name={surface_coat}\n')
        file.write(f'{elset_coat_tie}, SNEG\n')

        # define the constraint
        tie = 'TIE_CONSTRAINT'
        file.write(f'*Tie, name={tie}, adjust=yes\n')
        file.write(f'{surface_coat},{surface_volume_rough_top}\n')

        file.write('*End Assembly\n')

        #---------------------------------------------------------------
        # define two materials
        #---------------------------------------------------------------
        file.write(f"*Material, name={material_coat}\n")
        file.write("*Elastic\n")
        file.write("1., 0.3\n")

        file.write(f"*Material, name={material_volume}\n")
        file.write("*Elastic\n")
        file.write(f"{volume_young_moduli}, {volume_possion_ratio}\n")

        #---------------------------------------------------------------
        # Define STEP-X uniform tensile strain along X axis
        #---------------------------------------------------------------
        step_x = 'TENSILE_X'
        file.write(f"*Step, name={step_x}, nlgeom=NO\n")
        file.write("*Static\n")
        file.write("1., 1., 1e-05, 1.\n")
        file.write("**\n")

        # nset_vz_x0_y0 = 'VZ_X0_Y0'-node_volume_VZ_X0_Y0
        # nset_vz_x1_y0 = 'VZ_X1_Y0'-node_volume_VZ_X1_Y0
        # nset_vz_x0_y1 = 'VZ_X0_Y1'-node_volume_VZ_X0_Y1
        # nset_vz_x1_y1 = 'VZ_X1_Y1'-node_volume_VZ_X1_Y1
        # nset_ex_y0_z0 = 'EX_Y0_Z0'-nodes_volume_EX_Y0_Z0
        # nset_ex_y1_z0 = 'EX_Y1_Z0'-nodes_volume_EX_Y1_Z0
        # nset_ey_x0_z0 = 'EY_X0_Z0'-nodes_volume_EY_X0_Z0
        # nset_ey_x1_z0 = 'EY_X1_Z0'-nodes_volume_EY_X1_Z0
        # nset_ez_x0_y0 = 'EZ_X0_Y0'-nodes_volume_EZ_X0_Y0
        # nset_ez_x0_y1 = 'EZ_X0_Y1'-nodes_volume_EZ_X0_Y1
        # nset_ez_x1_y0 = 'EZ_X1_Y0'-nodes_volume_EZ_X1_Y0
        # nset_ez_x1_y1 = 'EZ_X1_Y1'-nodes_volume_EZ_X1_Y1
        # nset_x_negative_face = 'F_X0'-nodes_volume_X_negative
        # nset_x_positive_face = 'F_X1'-nodes_volume_X_positive
        # nset_y_negative_face = 'F_Y0'-nodes_volume_Y_negative
        # nset_y_positive_face = 'F_Y1'-nodes_volume_Y_positive
        # nset_z_bottom_face = 'F_Z0'-nodes_volume_Z_bottom
        #
        # Define boundary conditions
        #
        # strain amplitude
        strain_x = nom_stress/volume_young_moduli
        strain_y = - volume_possion_ratio * strain_x

        disp_x = strain_x * (x_1D_crop_struct_grids[-1] - x_1D_crop_struct_grids[0])
        disp_y = strain_y * (y_1D_crop_struct_grids[-1] - y_1D_crop_struct_grids[0])
        
        file.write(f"** boundary conditions\n")
        file.write("*Boundary, op=NEW\n")
        # V(0  0  0)
        file.write(f"{nset_vz_x0_y0}, 1, 1\n")
        file.write(f"{nset_vz_x0_y0}, 2, 2\n")
        file.write(f"{nset_vz_x0_y0}, 3, 3\n")
        # V(1  0  0)
        file.write(f"{nset_vz_x1_y0}, 1, 1, {disp_x}\n")
        file.write(f"{nset_vz_x1_y0}, 2, 2\n")
        file.write(f"{nset_vz_x1_y0}, 3, 3\n")
        # V(1  1  0)
        file.write(f"{nset_vz_x1_y1}, 1, 1, {disp_x}\n")
        file.write(f"{nset_vz_x1_y1}, 2, 2, {disp_y}\n")
        file.write(f"{nset_vz_x1_y1}, 3, 3\n")
        # V(0  1  0)
        file.write(f"{nset_vz_x0_y1}, 1, 1\n")
        file.write(f"{nset_vz_x0_y1}, 2, 2, {disp_y}\n")
        file.write(f"{nset_vz_x0_y1}, 3, 3\n")

        # E->X Y=0 Z=0
        for id_node in nodes_volume_EX_Y0_Z0:        
            #                                        V    V              
            disp = strain_x*(nodes_volume[id_node-1][0] - x_1D_crop_struct_grids[0]) # linear with X-coordinate
            file.write(f"{part_volume+'-1.'+str(id_node)}, 1, 1, {disp}\n")

        file.write(f"{nset_ex_y0_z0}, 2, 2\n")
        file.write(f"{nset_ex_y0_z0}, 3, 3\n")

        # E->X Y=1 Z=0
        for id_node in nodes_volume_EX_Y1_Z0:
            #                                        V    V              
            disp = strain_x*(nodes_volume[id_node-1][0] - x_1D_crop_struct_grids[0]) # linear with X-coordinate
            file.write(f"{part_volume+'-1.'+str(id_node)}, 1, 1, {disp}\n")

        file.write(f"{nset_ex_y1_z0}, 2, 2, {disp_y}\n")
        file.write(f"{nset_ex_y1_z0}, 3, 3\n")

        # E->Y X=0 Z=0
        file.write(f"{nset_ey_x0_z0}, 1, 1\n")
        for id_node in nodes_volume_EY_X0_Z0:
            #                                        V    V              
            disp = strain_y*(nodes_volume[id_node-1][1] - y_1D_crop_struct_grids[0])
            file.write(f"{part_volume+'-1.'+str(id_node)}, 2, 2, {disp}\n")

        file.write(f"{nset_ey_x0_z0}, 3, 3\n")
        
        # E->Y X=1 Z=0
        file.write(f"{nset_ey_x1_z0}, 1, 1, {disp_x}\n")
        for id_node in nodes_volume_EY_X1_Z0:
            #                                        V    V              
            disp = strain_y*(nodes_volume[id_node-1][1] - y_1D_crop_struct_grids[0])
            file.write(f"{part_volume+'-1.'+str(id_node)}, 2, 2, {disp}\n")

        file.write(f"{nset_ey_x1_z0}, 3, 3\n")

        # E->Z X=0 Y=0
        file.write(f"{nset_ez_x0_y0}, 1, 1\n")
        file.write(f"{nset_ez_x0_y0}, 2, 2\n")
        # E->Z X=1 Y=0
        file.write(f"{nset_ez_x1_y0}, 1, 1, {disp_x}\n")
        file.write(f"{nset_ez_x1_y0}, 2, 2\n")
        # E->Z X=1 Y=1
        file.write(f"{nset_ez_x1_y1}, 1, 1, {disp_x}\n")
        file.write(f"{nset_ez_x1_y1}, 2, 2, {disp_y}\n")
        # E->Z X=0 Y=1
        file.write(f"{nset_ez_x0_y1}, 1, 1\n")
        file.write(f"{nset_ez_x0_y1}, 2, 2, {disp_y}\n")

        # F-X=0
        file.write(f"{nset_x_negative_face}, 1, 1\n")
        for id_node in nodes_volume_X_negative:
            #                                        V    V              
            disp = strain_y*(nodes_volume[id_node-1][1] - y_1D_crop_struct_grids[0])
            file.write(f"{part_volume+'-1.'+str(id_node)}, 2, 2, {disp}\n")

        # F-X=1
        file.write(f"{nset_x_positive_face}, 1, 1, {disp_x}\n") # displacement along tensile direction
        for id_node in nodes_volume_X_positive:
            #                                        V    V              
            disp = strain_y*(nodes_volume[id_node-1][1] - y_1D_crop_struct_grids[0])
            file.write(f"{part_volume+'-1.'+str(id_node)}, 2, 2, {disp}\n")

        # F-Y=0
        for id_node in nodes_volume_Y_negative:
            #                                        V    V          
            disp = strain_x*(nodes_volume[id_node-1][0] - x_1D_crop_struct_grids[0]) # linear with X-coordinate
            file.write(f"{part_volume+'-1.'+str(id_node)}, 1, 1, {disp}\n")

        file.write(f"{nset_y_negative_face}, 2, 2\n")

        # F-Y=1
        for id_node in nodes_volume_Y_positive:
            #                                        V    V          
            disp = strain_x*(nodes_volume[id_node-1][0] - x_1D_crop_struct_grids[0]) # linear with X-coordinate
            file.write(f"{part_volume+'-1.'+str(id_node)}, 1, 1, {disp}\n")

        file.write(f"{nset_y_positive_face}, 2, 2, {disp_y}\n")

        # F-Z=0
        for id_node in nodes_volume_Z_bottom:
            #                                        V    V          
            disp = strain_x*(nodes_volume[id_node-1][0] - x_1D_crop_struct_grids[0]) # linear with X-coordinate
            file.write(f"{part_volume+'-1.'+str(id_node)}, 1, 1, {disp}\n")

        for id_node in nodes_volume_Z_bottom:
            #                                        V    V          
            disp = strain_y*(nodes_volume[id_node-1][1] - y_1D_crop_struct_grids[0]) # linear with X-coordinate
            file.write(f"{part_volume+'-1.'+str(id_node)}, 2, 2, {disp}\n")

        file.write(f"{nset_z_bottom_face}, 3, 3\n")

        file.write("*Restart, write, frequency=0\n")

        # only output E and S in the COAT (membrane)
        # file.write("*Output, field, frequency=99999\n")
        # file.write(f"*Element Output, elset={part_coat+'-1.'+elset_coat_sect}, directions=YES\n")
        # file.write("LE, S\n")
        # file.write("*Output, history, frequency=0\n")

        # output all the necessary variables
        file.write("*Output, field, variable=PRESELECT\n")
        file.write("*Output, history, variable=PRESELECT\n")

        # End step
        file.write("*End Step\n")

        #---------------------------------------------------------------
        # Define STEP-Y uniform tensile strain along X axis
        #---------------------------------------------------------------
        step_y = 'TENSILE_Y'
        file.write(f"*Step, name={step_y}, nlgeom=NO\n")
        file.write("*Static\n")
        file.write("1., 1., 1e-05, 1.\n")
        file.write("**\n")

        # strain amplitude
        strain_y = nom_stress/volume_young_moduli
        strain_x = - volume_possion_ratio * strain_y

        disp_x = strain_x * (x_1D_crop_struct_grids[-1] - x_1D_crop_struct_grids[0])
        disp_y = strain_y * (y_1D_crop_struct_grids[-1] - y_1D_crop_struct_grids[0])
        
        file.write(f"** boundary conditions\n")
        file.write("*Boundary, op=NEW\n")
        # V(0  0  0)
        file.write(f"{nset_vz_x0_y0}, 1, 1\n")
        file.write(f"{nset_vz_x0_y0}, 2, 2\n")
        file.write(f"{nset_vz_x0_y0}, 3, 3\n")
        # V(1  0  0)
        file.write(f"{nset_vz_x1_y0}, 1, 1, {disp_x}\n")
        file.write(f"{nset_vz_x1_y0}, 2, 2\n")
        file.write(f"{nset_vz_x1_y0}, 3, 3\n")
        # V(1  1  0)
        file.write(f"{nset_vz_x1_y1}, 1, 1, {disp_x}\n")
        file.write(f"{nset_vz_x1_y1}, 2, 2, {disp_y}\n")
        file.write(f"{nset_vz_x1_y1}, 3, 3\n")
        # V(0  1  0)
        file.write(f"{nset_vz_x0_y1}, 1, 1\n")
        file.write(f"{nset_vz_x0_y1}, 2, 2, {disp_y}\n")
        file.write(f"{nset_vz_x0_y1}, 3, 3\n")

        # E->X Y=0 Z=0
        for id_node in nodes_volume_EX_Y0_Z0:        
            #                                        V    V              
            disp = strain_x*(nodes_volume[id_node-1][0] - x_1D_crop_struct_grids[0]) # linear with X-coordinate
            file.write(f"{part_volume+'-1.'+str(id_node)}, 1, 1, {disp}\n")

        file.write(f"{nset_ex_y0_z0}, 2, 2\n")
        file.write(f"{nset_ex_y0_z0}, 3, 3\n")

        # E->X Y=1 Z=0
        for id_node in nodes_volume_EX_Y1_Z0:
            #                                        V    V              
            disp = strain_x*(nodes_volume[id_node-1][0] - x_1D_crop_struct_grids[0]) # linear with X-coordinate
            file.write(f"{part_volume+'-1.'+str(id_node)}, 1, 1, {disp}\n")

        file.write(f"{nset_ex_y1_z0}, 2, 2, {disp_y}\n")
        file.write(f"{nset_ex_y1_z0}, 3, 3\n")

        # E->Y X=0 Z=0
        file.write(f"{nset_ey_x0_z0}, 1, 1\n")
        for id_node in nodes_volume_EY_X0_Z0:
            #                                        V    V              
            disp = strain_y*(nodes_volume[id_node-1][1] - y_1D_crop_struct_grids[0])
            file.write(f"{part_volume+'-1.'+str(id_node)}, 2, 2, {disp}\n")

        file.write(f"{nset_ey_x0_z0}, 3, 3\n")
        
        # E->Y X=1 Z=0
        file.write(f"{nset_ey_x1_z0}, 1, 1, {disp_x}\n")
        for id_node in nodes_volume_EY_X1_Z0:
            #                                        V    V              
            disp = strain_y*(nodes_volume[id_node-1][1] - y_1D_crop_struct_grids[0])
            file.write(f"{part_volume+'-1.'+str(id_node)}, 2, 2, {disp}\n")

        file.write(f"{nset_ey_x1_z0}, 3, 3\n")

        # E->Z X=0 Y=0
        file.write(f"{nset_ez_x0_y0}, 1, 1\n")
        file.write(f"{nset_ez_x0_y0}, 2, 2\n")
        # E->Z X=1 Y=0
        file.write(f"{nset_ez_x1_y0}, 1, 1, {disp_x}\n")
        file.write(f"{nset_ez_x1_y0}, 2, 2\n")
        # E->Z X=1 Y=1
        file.write(f"{nset_ez_x1_y1}, 1, 1, {disp_x}\n")
        file.write(f"{nset_ez_x1_y1}, 2, 2, {disp_y}\n")
        # E->Z X=0 Y=1
        file.write(f"{nset_ez_x0_y1}, 1, 1\n")
        file.write(f"{nset_ez_x0_y1}, 2, 2, {disp_y}\n")

        # F-X=0
        file.write(f"{nset_x_negative_face}, 1, 1\n")
        for id_node in nodes_volume_X_negative:
            #                                        V    V              
            disp = strain_y*(nodes_volume[id_node-1][1] - y_1D_crop_struct_grids[0])
            file.write(f"{part_volume+'-1.'+str(id_node)}, 2, 2, {disp}\n")

        # F-X=1
        file.write(f"{nset_x_positive_face}, 1, 1, {disp_x}\n") # displacement along tensile direction
        for id_node in nodes_volume_X_positive:
            #                                        V    V              
            disp = strain_y*(nodes_volume[id_node-1][1] - y_1D_crop_struct_grids[0])
            file.write(f"{part_volume+'-1.'+str(id_node)}, 2, 2, {disp}\n")

        # F-Y=0
        for id_node in nodes_volume_Y_negative:
            #                                        V    V          
            disp = strain_x*(nodes_volume[id_node-1][0] - x_1D_crop_struct_grids[0]) # linear with X-coordinate
            file.write(f"{part_volume+'-1.'+str(id_node)}, 1, 1, {disp}\n")

        file.write(f"{nset_y_negative_face}, 2, 2\n")

        # F-Y=1
        for id_node in nodes_volume_Y_positive:
            #                                        V    V          
            disp = strain_x*(nodes_volume[id_node-1][0] - x_1D_crop_struct_grids[0]) # linear with X-coordinate
            file.write(f"{part_volume+'-1.'+str(id_node)}, 1, 1, {disp}\n")

        file.write(f"{nset_y_positive_face}, 2, 2, {disp_y}\n")

        # F-Z=0
        for id_node in nodes_volume_Z_bottom:
            #                                        V    V          
            disp = strain_x*(nodes_volume[id_node-1][0] - x_1D_crop_struct_grids[0]) # linear with X-coordinate
            file.write(f"{part_volume+'-1.'+str(id_node)}, 1, 1, {disp}\n")

        for id_node in nodes_volume_Z_bottom:
            #                                        V    V          
            disp = strain_y*(nodes_volume[id_node-1][1] - y_1D_crop_struct_grids[0]) # linear with X-coordinate
            file.write(f"{part_volume+'-1.'+str(id_node)}, 2, 2, {disp}\n")

        file.write(f"{nset_z_bottom_face}, 3, 3\n")        

        file.write("*Restart, write, frequency=0\n")

        # only output E and S in the COAT (membrane)
        # file.write("*Output, field, frequency=99999\n")
        # file.write(f"*Element Output, elset={part_coat+'-1.'+elset_coat_sect}, directions=YES\n")
        # file.write("LE, S\n")
        # file.write("*Output, history, frequency=0\n")

        # output all the necessary variables
        file.write("*Output, field, variable=PRESELECT\n")
        file.write("*Output, history, variable=PRESELECT\n")

        # End step
        file.write("*End Step\n")

        #---------------------------------------------------------------
        # Define STEP-shear uniform tensile strain along X axis
        #---------------------------------------------------------------
        step_shear = 'SHEAR'
        file.write(f"*Step, name={step_shear}, nlgeom=NO\n")
        file.write("*Static\n")
        file.write("1., 1., 1e-05, 1.\n")
        file.write("**\n")

        # strain amplitude
        gamma_xy = nom_stress/volume_shear_moduli

        disp_x = gamma_xy * (y_1D_crop_struct_grids[-1] - y_1D_crop_struct_grids[0])
        
        file.write(f"** boundary conditions\n")
        file.write("*Boundary, op=NEW\n")
        # V(0  0  0)
        file.write(f"{nset_vz_x0_y0}, 1, 1\n")
        file.write(f"{nset_vz_x0_y0}, 2, 2\n")
        file.write(f"{nset_vz_x0_y0}, 3, 3\n")
        # V(1  0  0)
        file.write(f"{nset_vz_x1_y0}, 1, 1\n")
        file.write(f"{nset_vz_x1_y0}, 2, 2\n")
        file.write(f"{nset_vz_x1_y0}, 3, 3\n")
        # V(1  1  0)
        file.write(f"{nset_vz_x1_y1}, 1, 1, {disp_x}\n")
        file.write(f"{nset_vz_x1_y1}, 2, 2\n")
        file.write(f"{nset_vz_x1_y1}, 3, 3\n")
        # V(0  1  0)
        file.write(f"{nset_vz_x0_y1}, 1, 1, {disp_x}\n")
        file.write(f"{nset_vz_x0_y1}, 2, 2\n")
        file.write(f"{nset_vz_x0_y1}, 3, 3\n")

        # E->X Y=0 Z=0
        file.write(f"{nset_ex_y0_z0}, 1, 1\n")
        file.write(f"{nset_ex_y0_z0}, 2, 2\n")
        file.write(f"{nset_ex_y0_z0}, 3, 3\n")

        # E->X Y=1 Z=0
        file.write(f"{nset_ex_y1_z0}, 1, 1, {disp_x}\n")
        file.write(f"{nset_ex_y1_z0}, 2, 2\n")
        file.write(f"{nset_ex_y1_z0}, 3, 3\n")

        # E->Y X=0 Z=0
        
        for id_node in nodes_volume_EY_X0_Z0:
            #                                        V    V              
            disp = gamma_xy*(nodes_volume[id_node-1][1] - y_1D_crop_struct_grids[0])
            file.write(f"{part_volume+'-1.'+str(id_node)}, 1, 1, {disp}\n")

        file.write(f"{nset_ey_x0_z0}, 2, 2\n")
        file.write(f"{nset_ey_x0_z0}, 3, 3\n")
        
        # E->Y X=1 Z=0
        
        for id_node in nodes_volume_EY_X1_Z0:
            #                                        V    V              
            disp = gamma_xy*(nodes_volume[id_node-1][1] - y_1D_crop_struct_grids[0])
            file.write(f"{part_volume+'-1.'+str(id_node)}, 1, 1, {disp}\n")

        file.write(f"{nset_ey_x1_z0}, 2, 2\n")
        file.write(f"{nset_ey_x1_z0}, 3, 3\n")

        # E->Z X=0 Y=0
        file.write(f"{nset_ez_x0_y0}, 1, 1\n")
        file.write(f"{nset_ez_x0_y0}, 2, 2\n")

        # E->Z X=1 Y=0
        file.write(f"{nset_ez_x1_y0}, 1, 1\n")
        file.write(f"{nset_ez_x1_y0}, 2, 2\n")

        # E->Z X=1 Y=1
        file.write(f"{nset_ez_x1_y1}, 1, 1, {disp_x}\n")
        file.write(f"{nset_ez_x1_y1}, 2, 2\n")

        # E->Z X=0 Y=1
        file.write(f"{nset_ez_x0_y1}, 1, 1, {disp_x}\n")
        file.write(f"{nset_ez_x0_y1}, 2, 2\n")

        # F-X=0
        for id_node in nodes_volume_X_negative:
            #                                        V    V              
            disp = gamma_xy*(nodes_volume[id_node-1][1] - y_1D_crop_struct_grids[0])
            file.write(f"{part_volume+'-1.'+str(id_node)}, 1, 1, {disp}\n")
            
        file.write(f"{nset_x_negative_face}, 2, 2\n")

        # F-X=1        
        for id_node in nodes_volume_X_positive:
            #                                        V    V              
            disp = gamma_xy*(nodes_volume[id_node-1][1] - y_1D_crop_struct_grids[0])
            file.write(f"{part_volume+'-1.'+str(id_node)}, 1, 1, {disp}\n")

        file.write(f"{nset_x_positive_face}, 2, 2\n")

        # F-Y=0
        file.write(f"{nset_y_negative_face}, 1, 1\n")
        file.write(f"{nset_y_negative_face}, 2, 2\n")

        # F-Y=1
        file.write(f"{nset_y_positive_face}, 1, 1, {disp_x}\n")
        file.write(f"{nset_y_positive_face}, 2, 2\n")

        # F-Z=0
        for id_node in nodes_volume_Z_bottom:
            #                                        V    V          
            disp = gamma_xy*(nodes_volume[id_node-1][1] - y_1D_crop_struct_grids[0]) # linear with X-coordinate
            file.write(f"{part_volume+'-1.'+str(id_node)}, 1, 1, {disp}\n")

        file.write(f"{nset_z_bottom_face}, 2, 2\n")     
        file.write(f"{nset_z_bottom_face}, 3, 3\n")     

        file.write("*Restart, write, frequency=0\n")

        # only output E and S in the COAT (membrane)
        # file.write("*Output, field, frequency=99999\n")
        # file.write(f"*Element Output, elset={part_coat+'-1.'+elset_coat_sect}, directions=YES\n")
        # file.write("LE, S\n")
        # file.write("*Output, history, frequency=0\n")

        # output all the necessary variables
        file.write("*Output, field, variable=PRESELECT\n")
        file.write("*Output, history, variable=PRESELECT\n")

        # End step
        file.write("*End Step\n")


        # finish writing inp file
        print(f'INP file has been written to {file_name_abaqus}')


In [30]:
def generate_abaqus_inp_file(file_name_gmsh, rand_surf_id):
    #-----------------------------------------------------------------------
    # Read the Abaqus input file and return the content as a list of strings.
    #-----------------------------------------------------------------------
    with open(file_name_gmsh, 'r') as file:
        lines = file.readlines()

    #-----------------------------------------------------------------------
    # Parse nodes and volume and surface elements
    #-----------------------------------------------------------------------
    nodes_volume = parse_nodes(lines)
    elements_volume, elements_surface = parse_elements(lines)

    #-----------------------------------------------------------------------
    # finds the nodes on left, right, front, back and bottom planes
    #-----------------------------------------------------------------------
    x0 = x_1D_crop_struct_grids[0]
    x1 = x_1D_crop_struct_grids[-1]

    y0 = y_1D_crop_struct_grids[0]
    y1 = y_1D_crop_struct_grids[-1]

    # extract node set in volume for applying boundary conditions
    tol = 1E-6
    # four vertice nodes
    node_volume_VZ_X0_Y0 = 0
    node_volume_VZ_X1_Y0 = 0
    node_volume_VZ_X0_Y1 = 0
    node_volume_VZ_X1_Y1 = 0

    # nodes inside the edge
    nodes_volume_EX_Y0_Z0 = []
    nodes_volume_EX_Y1_Z0 = []
    nodes_volume_EY_X0_Z0 = []
    nodes_volume_EY_X1_Z0 = []

    nodes_volume_EZ_X0_Y0 = []
    nodes_volume_EZ_X0_Y1 = []
    nodes_volume_EZ_X1_Y0 = []
    nodes_volume_EZ_X1_Y1 = []

    # nodes inside the faces
    nodes_volume_X_positive = []
    nodes_volume_X_negative = []

    nodes_volume_Y_positive = []
    nodes_volume_Y_negative = []

    nodes_volume_Z_bottom = []

    for index, node in enumerate(nodes_volume):
        X, Y, Z = node[0], node[1], node[2]
        if abs(Z) < tol:
            # Z=0
            if abs(X - x0) < tol and abs(Y - y0) < tol:
                # X=0 Y=0 Z=0
                node_volume_VZ_X0_Y0 = index + 1
            elif abs(X - x1) < tol and abs(Y - y0) < tol:
                # X=1 Y=0 Z=0
                node_volume_VZ_X1_Y0 = index + 1
            elif abs(X - x0) < tol and abs(node[1] - y1) < tol:
                # X=0 Y=1 Z=0
                node_volume_VZ_X0_Y1 = index + 1
            elif abs(X - x1) < tol and abs(node[1] - y1) < tol:
                # X=1 Y=1 Z=0
                node_volume_VZ_X1_Y1 = index + 1
            elif abs(X - x0) < tol:
                # X=0, Z=0
                nodes_volume_EY_X0_Z0.append(index + 1)
            elif abs(X - x1) < tol:
                # X=1, Z=0
                nodes_volume_EY_X1_Z0.append(index + 1)
            elif abs(Y - y0) < tol:
                # Y=0, Z=0
                nodes_volume_EX_Y0_Z0.append(index + 1)
            elif abs(Y - y1) < tol:
                # Y=1, Z=0
                nodes_volume_EX_Y1_Z0.append(index + 1)
            else:
                # inner nodes
                nodes_volume_Z_bottom.append(index + 1)

        else:
            # Z!=0
            if abs(X - x0) < tol and abs(Y - y0) < tol:
                # X=0, Y=0
                nodes_volume_EZ_X0_Y0.append(index + 1)
            elif abs(X - x1) < tol and abs(Y - y0) < tol:
                # X=1, Y=0
                nodes_volume_EZ_X1_Y0.append(index + 1)
            elif abs(X - x0) < tol and abs(node[1] - y1) < tol:
                # X=0, Y=1
                nodes_volume_EZ_X0_Y1.append(index + 1)
            elif abs(X - x1) < tol and abs(node[1] - y1) < tol:
                # X=1, Y=1
                nodes_volume_EZ_X1_Y1.append(index + 1)
            elif abs(X - x0) < tol:
                # X=0
                nodes_volume_X_negative.append(index + 1)
            elif abs(X - x1) < tol:
                # X=1
                nodes_volume_X_positive.append(index + 1)
            elif abs(Y - y0) < tol:
                # Y=0
                nodes_volume_Y_negative.append(index + 1)
            elif abs(Y - y1) < tol:
                # Y=1
                nodes_volume_Y_positive.append(index + 1)
            else:
                pass

        # if abs(node[0] - range_low_crop_struct_grids) < tol:
        #     nodes_volume_X_negative.append(index + 1)
        # if abs(node[0] - range_upp_crop_struct_grids) < tol:
        #     nodes_volume_X_positive.append(index + 1)

        # if abs(node[1] - range_low_crop_struct_grids) < tol:
        #     nodes_volume_Y_negative.append(index + 1)
        # if abs(node[1] - range_upp_crop_struct_grids) < tol:
        #     nodes_volume_Y_positive.append(index + 1)

        # if abs(node[2]) < tol:
        #     nodes_volume_Z_bottom.append(index + 1)

    #-----------------------------------------------------------------------
    # find the surface elements on the top rough surface
    #-----------------------------------------------------------------------
    elements_surface_Z_top = parse_top_surface_elements(elements_surface, nodes_volume, x0, x1)
    # print(len(elements_surface_Z_top))

    #-----------------------------------------------------------------------
    # find the nodes on the top surface and elements attached to the top surface
    # The C3D4 elements are catogerized into S1, S2, S3, S4 types
    #-----------------------------------------------------------------------
    nodes_set_surface, elements_volume_top_s1, elements_volume_top_s2, elements_volume_top_s3, elements_volume_top_s4 = volume_elements_neighboring_top_surface(elements_volume, elements_surface_Z_top)

    nodes_volume_Z_top = list(nodes_set_surface)

    #-----------------------------------------------------------------------
    # re-organize the nodes and surface elements on the top surface
    #-----------------------------------------------------------------------
    """
    Form the membrane elements on the rough surface.
    The dense of the element might not be enough.
    The membrane will be coupled with surface sets of the rough surface.
    """
    # X, Y, Z can form the grid of points on the rough surface
    # *the membrane has been cropped*
    nodes_coat_reindex = {}
    new_node_index = 1

    nodes_coat = []
    for node_id in nodes_set_surface:
        nodes_coat_reindex[node_id] = new_node_index
        new_node_index += 1
        node = nodes_volume[node_id-1]
        nodes_coat.append(node)


    
    # 重新编号 surface nodes，同时更新 elements_coat 中对应的结点编号
    # updated_elements_coat = []
    elements_coat = []
    new_element_index = 1
    for element in elements_surface_Z_top:
        updated_element = [new_element_index] + [nodes_coat_reindex[node_id] for node_id in element[1:]]  # 用新编号替换原来的结点编号
        elements_coat.append(updated_element)
        new_element_index += 1

    #-----------------------------------------------------------------------
    # write abaqus inp file and download it
    #-----------------------------------------------------------------------
    effective_crop_length = (number_crop_struct_grids-1)*interval_struct_grids

    model_name = f"abq_surface_L{round((range_upp_struct_grids - range_low_struct_grids)*1000)}um_C{num_crop_single_edge}_S{int(elem_sub_times)}_ID{rand_surf_id:0{5}d}"
    format_inp = "inp"
    file_name_abaqus = f"{model_name}.{format_inp}"



    # write inp file
    abaqus_inp_file_path = os.path.join(abaqus_inp_files_folder, file_name_abaqus)

    write_inp_file(nodes_volume, elements_volume, nodes_coat, elements_coat, nodes_volume_Z_top,
            node_volume_VZ_X0_Y0, node_volume_VZ_X1_Y0, node_volume_VZ_X0_Y1, node_volume_VZ_X1_Y1,
            nodes_volume_EX_Y0_Z0, nodes_volume_EX_Y1_Z0, nodes_volume_EY_X0_Z0, nodes_volume_EY_X1_Z0,
            nodes_volume_EZ_X0_Y0, nodes_volume_EZ_X0_Y1, nodes_volume_EZ_X1_Y0, nodes_volume_EZ_X1_Y1,
            nodes_volume_X_positive, nodes_volume_X_negative, nodes_volume_Y_positive, nodes_volume_Y_negative, nodes_volume_Z_bottom,
            elements_volume_top_s1, elements_volume_top_s2, elements_volume_top_s3, elements_volume_top_s4, abaqus_inp_file_path)

    # delete the gmsh file
    if os.path.exists(file_name_gmsh):
        os.remove(file_name_gmsh)
        print(f"File {file_name_gmsh} has been deleted.")

    # download the new inp file
    # files.download(file_name_abaqus)
    # return abaqus_inp_file_path

# Loop to generate a lot of ABAQUS inps

In [31]:
rand_seed_file = "random_seed.csv"

rand_seed_path = os.path.join(rand_seed_file_folder, rand_seed_file)
# Read the CSV file and save the values into a NumPy array
random_seeds = np.loadtxt(rand_seed_path, dtype=np.uint32, delimiter=',')

print(f"Loaded {len(random_seeds)} random seeds from {rand_seed_path}")
# print(random_seeds[0:2])

Loaded 100000 random seeds from C:\Abaqus_Works\roughness_simulation\random_seed.csv


In [32]:
id_job_start = 50000
id_job_end = 50001
print(f"Check if random SEED is used or not?")

b_seed = True

for id_surf in range(id_job_start, id_job_end, 1):
    print(f">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
    print(f"Random surface id: {id_surf:0{5}}")
    # generate random surface
    
    x_1D_crop_struct_grids, y_1D_crop_struct_grids, Z_2D_crop_struct_grids=generate_crop_random_structured_grids(id_surf, random_seeds[id_surf], b_seed)
    # time.sleep(10)

    effective_crop_length = (number_crop_struct_grids-1)*interval_struct_grids
    print(f"The length of the cropped structured grids is {round(effective_crop_length*1000)} um")

    # generate mesh by gmsh
    file_name_gmsh = generate_mesh_using_gmsh(x_1D_crop_struct_grids, y_1D_crop_struct_grids, Z_2D_crop_struct_grids, id_surf)

    # generate inp for ABAQUS computation
    generate_abaqus_inp_file(file_name_gmsh, id_surf)
    # time.sleep(10)
    print(f"<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<")

Check if random SEED is used or not?
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Random surface id: 50000
Deterministic seed is used!
mean value and Ra of the dense sampling is -0.01163897382759201 and 0.42461998898158426
x shift -356 and y shift 145
the sampling range in the X or Y axis
[-1.424     -1.4200125 -1.416025  -1.4120375 -1.40805   -1.4040625
 -1.400075  -1.3960875 -1.3921    -1.3881125 -1.384125  -1.3801375
 -1.37615   -1.3721625 -1.368175  -1.3641875 -1.3602    -1.3562125
 -1.352225  -1.3482375 -1.34425   -1.3402625 -1.336275  -1.3322875
 -1.3283    -1.3243125 -1.320325  -1.3163375 -1.31235   -1.3083625
 -1.304375  -1.3003875 -1.2964    -1.2924125 -1.288425  -1.2844375
 -1.28045   -1.2764625 -1.272475  -1.2684875 -1.2645    -1.2605125
 -1.256525  -1.2525375 -1.24855   -1.2445625 -1.240575  -1.2365875
 -1.2326    -1.2286125 -1.224625  -1.2206375 -1.21665   -1.2126625
 -1.208675  -1.2046875 -1.2007   



---



Calculate the 1st order derivatives of the roughness profile

In [ ]:

full_length_mm = 0.636
num_crop_single_edge = 0
elem_sub_times = 1.0
interval_struct_grids = 0.004

id_job_start = 1000
id_job_end = 1200

number_struct_grids = 160

x_1D_struct_grids = np.linspace(0, full_length_mm, number_struct_grids)
y_1D_struct_grids = np.linspace(0, full_length_mm, number_struct_grids)

print(x_1D_struct_grids)

for id_surf in range(id_job_start, id_job_end, 1):
    print(f">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
    print(f"surface id: {id_surf:0{5}}")

    file_name_csv = f"input_surface_L{round(full_length_mm*1000)}um_C{num_crop_single_edge}_S{int(elem_sub_times)}_ID{id_surf:0{5}d}.csv"
    rough_surface_file_path_csv = os.path.join(input_rough_surface_folder, file_name_csv)

    # Read the CSV file
    R_val = pd.read_csv(rough_surface_file_path_csv, header=None).values  # Assuming no headers in the CSV
    
    # Create a RectBivariateSpline object
    spline = RectBivariateSpline(y_1D_struct_grids, x_1D_struct_grids, R_val)
    
    # Calculate first-order derivatives
    base_surf  = spline(x_1D_struct_grids, y_1D_struct_grids)
    dx_val     = spline(x_1D_struct_grids, y_1D_struct_grids, dx=1, dy=0)  # Derivative along X-axis
    dy_val     = spline(x_1D_struct_grids, y_1D_struct_grids, dx=0, dy=1)  # Derivative along Y-axis

    # x = np.arange(R_val.shape[1])  # X-axis grid points
    # y = np.arange(R_val.shape[0])  # Y-axis grid points
    # spline = RectBivariateSpline(y, x, R_val)
    # base_surf  = spline(y, x)
    # dx_val     = spline(y, x, dx=1, dy=0)  # Derivative along X-axis
    # dy_val     = spline(y, x, dx=0, dy=1)  # Derivative along Y-axis
    
    # Save original data and derivatives to Excel
    file_name_xlsx = f"input_surface_L{round(full_length_mm*1000)}um_C{num_crop_single_edge}_S{int(elem_sub_times)}_ID{id_surf:0{5}d}.xlsx"
    rough_surface_file_path_xlsx = os.path.join(input_rough_surface_folder, file_name_xlsx)

    with pd.ExcelWriter(rough_surface_file_path_xlsx, engine='openpyxl') as writer:
        pd.DataFrame(R_val).to_excel(writer, sheet_name=f"R", index=False, header=False)
        pd.DataFrame(dx_val).to_excel(writer, sheet_name=f"dRdx", index=False, header=False)
        pd.DataFrame(dy_val).to_excel(writer, sheet_name=f"dRdy", index=False, header=False)
    
    #--------------------------------------------------------------------------------------
    # fig, axes = plt.subplots(1, 2, figsize=(12, 6))  # Create 1 row, 2 columns of subplots
    # # Plot R_val
    # im1 = axes[0].imshow(R_val, cmap='jet', extent=[0, full_length_mm, 0, full_length_mm])
    # axes[0].set_title("Original Surface (R_val)")
    # axes[0].set_xlabel("X (mm)")
    # axes[0].set_ylabel("Y (mm)")
    # fig.colorbar(im1, ax=axes[0], orientation='vertical', label="Height")

    # # Plot base_surf
    # im2 = axes[1].imshow(dx_val, cmap='jet', extent=[0, full_length_mm, 0, full_length_mm])
    # axes[1].set_title("Interpolated Surface (base_surf)")
    # axes[1].set_xlabel("X (mm)")
    # axes[1].set_ylabel("Y (mm)")
    # fig.colorbar(im2, ax=axes[1], orientation='vertical', label="Height")

    # # Plot base_surf
    # # im3 = axes[2].imshow(base_surf-R_val, cmap='viridis', extent=[0, full_length_mm, 0, full_length_mm])
    # # axes[2].set_title("delta Surface ")
    # # axes[2].set_xlabel("X (mm)")
    # # axes[2].set_ylabel("Y (mm)")
    # # fig.colorbar(im3, ax=axes[2], orientation='vertical', label="Height")

    # # Adjust layout and show the plot
    # plt.tight_layout()
    # plt.show()

    # break
    print(f"<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<")


[0.    0.004 0.008 0.012 0.016 0.02  0.024 0.028 0.032 0.036 0.04  0.044
 0.048 0.052 0.056 0.06  0.064 0.068 0.072 0.076 0.08  0.084 0.088 0.092
 0.096 0.1   0.104 0.108 0.112 0.116 0.12  0.124 0.128 0.132 0.136 0.14
 0.144 0.148 0.152 0.156 0.16  0.164 0.168 0.172 0.176 0.18  0.184 0.188
 0.192 0.196 0.2   0.204 0.208 0.212 0.216 0.22  0.224 0.228 0.232 0.236
 0.24  0.244 0.248 0.252 0.256 0.26  0.264 0.268 0.272 0.276 0.28  0.284
 0.288 0.292 0.296 0.3   0.304 0.308 0.312 0.316 0.32  0.324 0.328 0.332
 0.336 0.34  0.344 0.348 0.352 0.356 0.36  0.364 0.368 0.372 0.376 0.38
 0.384 0.388 0.392 0.396 0.4   0.404 0.408 0.412 0.416 0.42  0.424 0.428
 0.432 0.436 0.44  0.444 0.448 0.452 0.456 0.46  0.464 0.468 0.472 0.476
 0.48  0.484 0.488 0.492 0.496 0.5   0.504 0.508 0.512 0.516 0.52  0.524
 0.528 0.532 0.536 0.54  0.544 0.548 0.552 0.556 0.56  0.564 0.568 0.572
 0.576 0.58  0.584 0.588 0.592 0.596 0.6   0.604 0.608 0.612 0.616 0.62
 0.624 0.628 0.632 0.636]
>>>>>>>>>>>>>>>>>>>>>>>>>>>>